In [1]:
import json
import requests
import sys
import warnings
from datetime import datetime, time, timedelta
from selenium import webdriver
GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
sys.path.insert(0, '../../../ajna_docs/commons/')
sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')
from virasana.integracao.due import raspa_due

warnings.simplefilter("ignore")

VIRASANA_URL = "https://ajna.labin.rf08.srf/virasana/"

2020-08-25 11:28:45,044 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../commons\ajna_commons\flask\access.log


In [2]:
def do_update(driver, tipo_manifesto):
    if tipo_manifesto == 'VAZIO':
            conteineres_ids = raspa_due.raspa_containers_vazios_sem_due(
            datainicial, datafinal)
    else:
        conteineres_ids = raspa_due.raspa_containers_sem_due(
            datainicial, datafinal, tipo_manifesto)
    print('%s contêineres a consultar em POS ACD' % len(conteineres_ids))
    conteineres_listadue = raspa_due.get_dues_pos_acd(driver, list(conteineres_ids.keys()))
    setdues = set()
    for conteiner, dues in conteineres_listadue.items():
        for due in dues:
            setdues.add(due)
    print('%s contêineres a consultar em DUE DETALHE' % len(setdues))
    due_detalhe = raspa_due.detalha_dues(driver, conteineres_listadue)
    print(len(due_detalhe))
    pacote_carregamento = {}
    for conteiner, numeros_dues in conteineres_listadue.items():
        _id = conteineres_ids[conteiner]
        lista_dues = []
        for due in numeros_dues:
            if due is not None:
                pacote = raspa_due.monta_due_ajna(due_detalhe[due])
                lista_dues.append({'numero': due, **pacote})
        if numeros_dues and len(numeros_dues) > 0:
            pacote_carregamento[_id] = lista_dues
    print('Inserindo %d dues...' % len(pacote_carregamento))
    r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento, verify=False)
    print(r.status_code)
    print(r.text)

In [7]:
diaapesquisar = datetime.today() - timedelta(days=3)
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2020-08-22  00:00:00 2020-08-22 23:59:59


In [ ]:
driver = webdriver.Chrome(GECKO_PATH)
try:
    raspa_due.auth_suite_rfb(driver)
    do_update(driver, 'lce')
    do_update(driver, None)
    do_update(driver, 'VAZIO')
finally:
    driver.close()


Conectando virasana
11 Contêineres recuperados.
11 contêineres a consultar em POS ACD
3 contêineres a consultar em DUE DETALHE
3
Inserindo 3 dues...
201
{"status":"DUEs inseridas/atualizadas"}

Conectando virasana
1645 Contêineres recuperados.
1550 contêineres a consultar em POS ACD


In [ ]:
params = {'query':
               {'metadata.carga.conhecimento.conhecimento': '152005011610119',
                 'metadata.contentType': 'image/jpeg',
                'metadata.numeroinformado': 'PONU7791526'
                 },
          'projection':
               {'_id': 1}
         }
r = requests.post('https://ajna.labin.rf08.srf/virasana/grid_data', json=params, verify=False)


In [ ]:
r.content

In [ ]:
lista = r.json()

In [ ]:
_id = lista[0]['_id']

In [ ]:
_id